In [16]:
# !pip install -qU pinecone openai
import os
from dotenv import load_dotenv
load_dotenv()  # .env 파일의 환경변수 로드

# Pinecone 연결 및 인덱스 초기화
import pinecone
from pinecone import Pinecone as PineconeClient

# API 키와 환경명 가져오기
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [17]:
from pinecone import Pinecone, ServerlessSpec

# API 키 설정 (본인의 API 키로 교체하세요)
pcone = Pinecone(api_key=PINECONE_API_KEY)   # Pinecone API 키 입력

# Pinecone 인덱스 생성
index_name = "movie-index"  # 인덱스 이름 설정
if not pcone.has_index(index_name):
    pcone.create_index(
        name=index_name,
        dimension=1536,            # 임베딩 벡터 차원 (모델 output 크기와 맞춤)
        metric="cosine",           # 유사도 metric 선택
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1"
        )
    )
# 생성된 인덱스에 연결
index = pcone.Index(index_name)

In [18]:
import pandas as pd

# 예시를 위한 간단한 데이터프레임 생성 (실제로는 CSV를 로드)
# df = pd.read_csv("KoreanDramas.csv")  # 실제 데이터셋 사용 시
data = [
    {
        "title": "응답하라 1988",
        "year": 2015,
        "genre": ["드라마", "코미디"],
        "director": "신원호",
        "actors": ["혜리", "박보검", "류준열"],
        "rating": 9.2,
        "synopsis": "1988년 서울, 쌍문동 이웃들 사이의 우정과 가족애를 그린 드라마."
    },
    {
        "title": "기생충",
        "year": 2019,
        "genre": ["드라마", "스릴러"],
        "director": "봉준호",
        "actors": ["송강호", "이선균", "조여정"],
        "rating": 8.6,
        "synopsis": "가난한 가족과 부유한 가족 사이 벌어지는 블랙코미디 풍자의 스릴러 영화."
    }
]
df = pd.DataFrame(data)
df.head(2)  # 데이터 확인 (실제로는 전체 데이터 사용)

,title,year,genre,director,actors,rating,synopsis
0,응답하라 1988,2015,"[드라마, 코미디]",신원호,"[혜리, 박보검, 류준열]",9.2,"1988년 서울, 쌍문동 이웃들 사이의 우정과 가족애를 그린 드라마."
1,기생충,2019,"[드라마, 스릴러]",봉준호,"[송강호, 이선균, 조여정]",8.6,가난한 가족과 부유한 가족 사이 벌어지는 블랙코미디 풍자의 스릴러 영화.


In [25]:
# 쿼리 문서 생성
data = [
    {
        "title": "응답하라 1988",
        "year": 2015,
        "genre": ["드라마", "코미디"],
        "director": "신원호",
        "actors": ["혜리", "박보검", "류준열"],
        "rating": 9.2,
        "synopsis": "1988년 서울, 쌍문동 이웃들 사이의 우정과 가족애를 그린 드라마."
    },
    {
        "title": "도깨비",
        "year": 2016,
        "genre": ["판타지", "로맨스", "드라마"],
        "director": "이응복",
        "actors": ["공유", "김고은", "이동욱", "유인나"],
        "rating": 8.9,
        "synopsis": "불멸의 삶을 사는 도깨비와 기억을 잃은 저승사자, 그리고 도깨비 신부의 운명적 만남을 그린 판타지 로맨스."
    },
    {
        "title": "태양의 후예",
        "year": 2016,
        "genre": ["로맨스", "드라마", "액션"],
        "director": "이응복",
        "actors": ["송중기", "송혜교", "진구", "김지원"],
        "rating": 8.7,
        "synopsis": "평화유지군 대위와 의사 사이의 국경을 초월한 사랑을 그린 로맨스 드라마."
    },
    {
        "title": "SKY 캐슬",
        "year": 2018,
        "genre": ["드라마", "블랙코미디"],
        "director": "조현탁",
        "actors": ["염정아", "이태란", "윤세아", "오나라"],
        "rating": 8.5,
        "synopsis": "상위 0.1% 상류층 부모들의 치열한 교육열과 그 뒤에 숨겨진 욕망을 그린 블랙코미디."
    },
    {
        "title": "사랑의 불시착",
        "year": 2019,
        "genre": ["로맨스", "드라마", "코미디"],
        "director": "이정효",
        "actors": ["현빈", "손예진", "서지혜", "김정현"],
        "rating": 8.8,
        "synopsis": "재벌 상속녀가 패러글라이딩 사고로 북한에 불시착하면서 벌어지는 로맨스 드라마."
    },
    {
        "title": "이태원 클라쓰",
        "year": 2020,
        "genre": ["드라마", "로맨스"],
        "director": "김성윤",
        "actors": ["박서준", "김다미", "유재명", "권나라"],
        "rating": 8.4,
        "synopsis": "이태원의 작은 술집에서 시작된 젊은 사업가의 성공과 복수, 그리고 사랑 이야기."
    },
    {
        "title": "킹덤",
        "year": 2019,
        "genre": ["스릴러", "호러", "사극"],
        "director": "김성훈",
        "actors": ["주지훈", "배두나", "류승룡", "김상호"],
        "rating": 8.3,
        "synopsis": "조선시대를 배경으로 한 좀비 창궐과 왕권 쟁탈전을 그린 스릴러 사극."
    },
    {
        "title": "펜트하우스",
        "year": 2020,
        "genre": ["드라마", "스릴러", "복수"],
        "director": "주동민",
        "actors": ["이지아", "김소연", "유진", "엄기준"],
        "rating": 7.9,
        "synopsis": "상류층 부모들의 자녀 교육 성공을 위한 욕망이 빚어내는 복수와 반전의 드라마."
    },
    {
        "title": "오징어 게임",
        "year": 2021,
        "genre": ["스릴러", "드라마", "서바이벌"],
        "director": "황동혁",
        "actors": ["이정재", "박해수", "위하준", "정호연"],
        "rating": 8.2,
        "synopsis": "거액의 상금을 두고 어린 시절 놀이를 소재로 한 생존 게임에 참가한 사람들의 이야기."
    },
    {
        "title": "미스터 션샤인",
        "year": 2018,
        "genre": ["사극", "로맨스", "드라마"],
        "director": "이응복",
        "actors": ["이병헌", "김태리", "유연석", "김민정"],
        "rating": 8.6,
        "synopsis": "1900년대 초 격동의 조선을 배경으로 한 미국 군인과 조선 여인의 애절한 사랑 이야기."
    }
]

df = pd.DataFrame(data)
df.head(2)  # 데이터 확인 (실제로는 전체 데이터 사용)

,title,year,genre,director,actors,rating,synopsis
0,응답하라 1988,2015,"[드라마, 코미디]",신원호,"[혜리, 박보검, 류준열]",9.2,"1988년 서울, 쌍문동 이웃들 사이의 우정과 가족애를 그린 드라마."
1,도깨비,2016,"[판타지, 로맨스, 드라마]",이응복,"[공유, 김고은, 이동욱, 유인나]",8.9,"불멸의 삶을 사는 도깨비와 기억을 잃은 저승사자, 그리고 도깨비 신부의 운명적 만남..."


In [26]:
# 임베딩 할 문장 리스트 준비 (각 아이템의 synopsis 텍스트)
texts = df["synopsis"].tolist()

In [27]:
from langchain_openai import OpenAIEmbeddings

openai_embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small",  # OpenAI 임베딩 모델 선택
    api_key=OPENAI_API_KEY  # OpenAI API 키 설정
)

In [ ]:
texts = list()
metadatas = list()
ids = list()

for i, row in df.iterrows():
    text = row["synopsis"]  # 각 영화의 시놉시스 텍스트
    if not isinstance(text, str):
        text = str(text)  # 텍스트가 문자열이 아닐 경우 문자열로 변환
    metadata = {
        "title": row["title"],
        "year": row["year"],
        "genre": row["genre"],
        "director": row["director"],
        "actors": row["actors"],
        "rating": row["rating"],
        "synopsis": text  # 메타데이터로 영화 정보 추가
    }
    ids.append(f"movie-{i+index.describe_index_stats()['total_vector_count'] }")  # 각 영화에 대한 고유 ID 생성
    texts.append(text)
    metadatas.append(metadata)  # 메타데이터 추가

print(f"총 {len(texts)}개의 영화 시놉시스가 준비되었습니다.")

총 10개의 영화 시놉시스가 준비되었습니다.


In [35]:
# 임베딩 벡터 생성
vectors = openai_embeddings.embed_documents(texts)

len(vectors)  # 생성된 벡터의 일부 확인

# type(vectors)

10

In [40]:
# pinecone에 벡터 업로드
index.upsert(
    vectors=zip(ids, vectors, metadatas),  # ID, 벡터, 메타데이터를 함께 업로드
    namespace="movies"  # 네임스페이스 설정 (선택 사항)
)
# 업로드된 벡터 개수 확인
print(f"업로드된 벡터 개수: {index.describe_index_stats()['total_vector_count']}")


업로드된 벡터 개수: 15


In [38]:
# 검색 쿼리 예시
query = "1980년대 서울을 배경으로 한 드라마"
# 쿼리 임베딩 생성
query_embedding = openai_embeddings.embed_query(query)
# 유사도 검색 수행
results = index.query(
    vector=query_embedding,  # 쿼리 벡터
    top_k=2,  # 상위 5개 결과 반환
    include_metadata=True,  # 메타데이터 포함
    namespace="movies"  # 네임스페이스 설정
)
# 검색 결과 출력
for match in results["matches"]:
    print(f"Title: {match['metadata']['title']}, Year: {match['metadata']['year']}, Score: {match['score']:.4f}")
    print(f"Synopsis: {match['metadata']['synopsis']}\n")

Title: 응답하라 1988, Year: 2015.0, Score: 0.6379
Synopsis: 1988년 서울, 쌍문동 이웃들 사이의 우정과 가족애를 그린 드라마.

Title: 응답하라 1988, Year: 2015.0, Score: 0.6374
Synopsis: 1988년 서울, 쌍문동 이웃들 사이의 우정과 가족애를 그린 드라마.

